In [3]:
import pandas as pd
import json
import datetime
import numpy as np
import math

In [2]:
path = "/Users/hogni/Library/Mobile Documents/com~apple~CloudDocs/Bachelor Project/Videos/Test/Data/"
columns = ["frame_id", "timestamp", "counter_area", "ObjectClass", "UniqueID", "bearing", "CountingDirection", "Angle"]
counter_df = pd.read_csv(f"{path}counterData-2021-03-10-60488f6516752f0054c6f771.csv", names=columns)

In [3]:
counter_df = counter_df[counter_df.duplicated(subset=["UniqueID"], keep=False)].sort_values("UniqueID")

In [4]:
counter_df['timestamp'] = pd.to_datetime(counter_df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%fZ')

In [5]:
def calculate_speed(distance, df):
    speed_dictionary = {}
    for index, row in df.iterrows():
        if row[4] in speed_dictionary.keys():
            speed_dictionary[row[4]].append(row[1])
        else:
            speed_dictionary[row[4]] = [row[1]]

    for key, value in speed_dictionary.copy().items():
        if value[0] > value[1]:
            difference = (value[0]-value[1])
        else:
            difference = (value[1]-value[0])
        speed_dictionary[key] = difference

    for key, value in speed_dictionary.copy().items():
        t = speed_dictionary[key].total_seconds()
        if t == 0:
            t = 1
            speed_dictionary[key] = distance/t
        else:
            speed_dictionary[key] = distance/t
    return speed_dictionary

In [6]:
speed = calculate_speed(1, counter_df)

for key, value in speed.items():
    counter_df.loc[counter_df["UniqueID"]==key, "velocity_m/s"]=value

In [7]:
def calculate_true_heading(frame_orientation, bearing):
    true_heading = bearing + frame_orientation
    if true_heading > 360:
        true_heading = true_heading - 360
    return true_heading

In [110]:
counter_df.head(6)

,frame_id,timestamp,counter_area,ObjectClass,UniqueID,bearing,CountingDirection,Angle,velocity_m/s
0,181,2021-03-10 09:20:58.348,start,bicycle,1454,92.161079,leftright_topbottom,29.842766,8.130081
1,182,2021-03-10 09:20:58.471,end,bicycle,1454,93.366461,leftright_topbottom,36.530463,8.130081
2,208,2021-03-10 09:21:01.498,end,bicycle,1458,92.726311,leftright_topbottom,35.890313,1.000000
3,208,2021-03-10 09:21:01.498,start,bicycle,1458,92.726311,leftright_topbottom,30.407997,1.000000
4,216,2021-03-10 09:21:02.407,start,bicycle,1459,92.436648,leftright_topbottom,30.118335,4.048583
5,218,2021-03-10 09:21:02.654,end,bicycle,1459,93.576334,leftright_topbottom,36.740337,4.048583


# Tensorflow FNN - Predict path.

In [100]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [116]:
X_vector_list = []
temp_vector = []
temp_id = 0
for index, row in counter_df.iterrows():
    if temp_id == 0:
        temp_id = row["UniqueID"]
        temp_vector.append(row["bearing"])
    else:
        if row["UniqueID"] == temp_id:
            temp_vector.append(row["bearing"])
            temp_vector.append(row["velocity_m/s"])
    if len(temp_vector) == 3:
        X_vector_list.append(temp_vector)
        temp_vector = []
        temp_id = 0
        
y_vector_list = [1 ,1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X_vector_list, y_vector_list, test_size=0.33, random_state=42)

In [118]:
depth = 5
y_train = tf.one_hot(y_train, depth)
y_test = tf.one_hot(y_test, depth)

In [119]:
X_train = tf.constant(X_train, dtype=tf.float32)
X_test = tf.constant(X_test, dtype=tf.float32)

In [120]:
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)),
  tf.keras.layers.Dense(20, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(5)
])

In [121]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [122]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [125]:
model.fit(X, y, epochs=5)

Epoch 1/5
1/1 [==============================] - 0s 2ms/step - loss: 78.1424 - accuracy: 0.1176
Epoch 2/5
1/1 [==============================] - 0s 2ms/step - loss: 89.1179 - accuracy: 0.0588
Epoch 3/5
1/1 [==============================] - 0s 2ms/step - loss: 84.7066 - accuracy: 0.1176
Epoch 4/5
1/1 [==============================] - 0s 2ms/step - loss: 85.6896 - accuracy: 0.0588
Epoch 5/5
1/1 [==============================] - 0s 2ms/step - loss: 86.9990 - accuracy: 0.0588


In [126]:
model.evaluate(X_test,  y_test, verbose=2)

1/1 - 0s - loss: 38.0700 - accuracy: 0.0000e+00


[38.0699577331543, 0.0]

# Calculate end co-ordinate

In [54]:
def end_coordinate(start_coordinates, bearing, speed):
    distance = speed
    departure = distance * math.sin(math.radians(bearing))
    latitude = distance * math.cos(math.radians(bearing))
    return (round((start_coordinates[0] + departure)), round((start_coordinates[1] + latitude)))